In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

# inversion model: wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
# cross-encoder: wandb.ai/jack-morris/emb-rerank-1/runs/7bd186119e1f4f789eaa0731fd7357a1/overview?workspace=user-jxmorris12
checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/98b1418d38c3f9333b17ab20bff06ff9/"

trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================


In [11]:
frozen_embeddings.shape

torch.Size([1, 768])

In [27]:
import torch

model = trainer.inversion_trainer.model
model.use_frozen_embeddings_as_input = True

fake_embedder_input_ids = torch.ones([1], dtype=torch.long, device=trainer.args.device)
fake_embedder_attention_mask = torch.ones([1], dtype=torch.long, device=trainer.args.device)

tt = model.embedder_tokenizer('This is a random sentence!', return_tensors='pt')
tt = tt.to(trainer.args.device)
print('goal text:', model.embedder_tokenizer.decode(tt['input_ids'][0]))

with torch.no_grad():
    original_embeddings = model.call_embedding_model(
        **tt
    )
    

#
# frozen_embeddings = torch.randn((1, 768), device=trainer.args.device, requires_grad=True)
frozen_embeddings = original_embeddings.clone()
frozen_embeddings.requires_grad = True
opt = torch.optim.Adam(params=[frozen_embeddings], lr=1e-4)
#

num_steps = 1000
log_interval = 20
for step in range(num_steps):
    output = model.forward(
        embedder_input_ids=fake_embedder_input_ids,
        embedder_attention_mask=fake_embedder_attention_mask,
        frozen_embeddings=frozen_embeddings,
        labels=tt['input_ids']
    )
    gen_tokens = model.generate(
        inputs={
            'embedder_input_ids': fake_embedder_input_ids,
            'embedder_attention_mask': fake_embedder_attention_mask,
            'frozen_embeddings': frozen_embeddings,
        },
        generation_kwargs={
            "max_length": 32,
            "early_stopping": True,
            "num_beams": 1,
            "do_sample": False,
        },
    )
    
    lm_loss = -1.0 * output.loss / 100.0
    cos_loss = 1.0 - torch.nn.CosineSimilarity(dim=1)(frozen_embeddings, original_embeddings)
    
    loss = (lm_loss + cos_loss)
    gen_text = model.embedder_tokenizer.decode(gen_tokens[0])
    if step % log_interval == 0:
        print(f'step {step} loss {loss} (lm_loss={lm_loss.item():.2f} cos_loss={cos_loss.item():.2f})')
        print(f'\tgen_text = {gen_text}')
    
    loss.backward()
    opt.step()

goal text: This is a random sentence!</s>
step 0 loss tensor([-0.0259], device='cuda:0', grad_fn=<AddBackward0>) (lm_loss=-0.03 cos_loss=0.00)
	gen_text = <pad> This is a random sentence! This is a random sentence! This is a random sentence! This is a random sentence! This is random
step 20 loss tensor([-0.0406], device='cuda:0', grad_fn=<AddBackward0>) (lm_loss=-0.04 cos_loss=0.00)
	gen_text = <pad> This Random Random Word is a Random & Random Task This Random Word is a Random Task and this is a random sentence from a baby
step 40 loss tensor([-0.0539], device='cuda:0', grad_fn=<AddBackward0>) (lm_loss=-0.06 cos_loss=0.00)
	gen_text = <pad> This Random Random House and This Random Child is a Random Task This Random House and This Random Child is a random sentence that is a sentence.
step 60 loss tensor([-0.0735], device='cuda:0', grad_fn=<AddBackward0>) (lm_loss=-0.08 cos_loss=0.00)
	gen_text = <pad> This Random Random House Is a Random Child This Random House Is a Random Child This R

In [21]:
torch.nn.CosineSimilarity(dim=1)(frozen_embeddings, original_embeddings)

tensor([-0.0275], device='cuda:0', grad_fn=<SumBackward1>)